## Librerie e funzioni

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Parametri geometrici delle lastre
larghezza_pla = 26.7  # cm
lunghezza_pla = 52.5  # cm
profondità_pla = 2.3  # cm

# Distanze tra i PMT
distanza_7_4 = 28.9  # cm
distanza_7_5=18.5 #cm
distanza_5_4=9.4 #cm
distanza_4_3=9.4 #cm
distanza_4_2 = 39.4  # cm
distanza_2_b = 7  # cm
distanza_b_1=6

# Efficienze dei PMT
efficienze = {
    "PMT7": 0.35,
    "PMT4": 0.729,
    "PMT2": 0.3,
    "PMT1": 0.15,
    "PMT8": 0.3,
    "PMT9": 0.4,
    "PMT10": 0.3,
    "PMT11": 0.4
}

# Dimensioni del bersaglio (PMT sotto il sistema principale)
larghezza_bers = 30  # cm
lunghezza_bers = 31  # cm

# Dimensioni di ciascun PMT nel bersaglio
larghezza_pmt_bers = larghezza_bers / 2  # ogni PMT largo metà del sistema
lunghezza_pmt_bers = lunghezza_bers
profondità_pmt_bers=30/2

offset_x=(lunghezza_pla-lunghezza_bers)/2
offset_y=(larghezza_bers-larghezza_pla)/2


## Toy MonteCarlo START

In [6]:
# Funzione per verificare se una posizione è all'interno dei limiti geometrici
def is_inside(x, y, lunghezza,larghezza):
    return (0<= x <= lunghezza) and (0 <= y <= larghezza)

"""
# Funzione per verificare se una posizione è all'interno dei limiti geometrici
def is_inside_target(x, y, lunghezza,larghezza):
    return (offset_x<= x <= lunghezza+offset_x) and (-offset_y <= y <= larghezza)
    """

# Funzione per verificare se una posizione è all'interno dei limiti geometrici considerando diverse condizioni degli scintillatori del bersaglio
def is_inside_target(pmt, x, y, lunghezza, larghezza):
    if pmt in ["PMT9", "PMT11"]:
        return (offset_x <= x <= lunghezza + offset_x) and (-offset_y <= y <= larghezza-offset_y)
    elif pmt in ["PMT8", "PMT10"]:
        return (offset_x <= x <= lunghezza + offset_x) and (larghezza-offset_y <= y <= 2*larghezza-offset_y)
    else:
        # Default logic
        return (offset_x <= x <= lunghezza + offset_x) and (-offset_y <= y <= larghezza-offset_y)
    
# Simulazione di un evento per i PMT bersaglio
def simulate_target_event(xb89, yb89, theta, phi):
    # Limiti del sistema bersaglio rispetto a PMT2    
    xb01 = xb89 - profondità_pmt_bers * np.tan(theta) * np.cos(phi)
    yb01 = yb89 - profondità_pmt_bers * np.tan(theta) * np.sin(phi)

    # Coordinate dei centri dei PMT bersaglio
    pmt_positions = {
        "PMT8": (xb89, yb89),
        "PMT9": (xb89, yb89),
        "PMT10": (xb01, yb01),
        "PMT11": (xb01, yb01), 
    }

    # Verifica hit su ciascun PMT
    hit_results = []
    for pmt, (px, py) in pmt_positions.items():
        is_hit = is_inside_target(pmt, px, py, lunghezza_pmt_bers,larghezza_pmt_bers) and np.random.uniform(0, 1) < efficienze[pmt]
        hit_results.append(is_hit)
    # Logica OR tra i PMT del bersaglio
    return any(hit_results)

# Simulazione di un evento principale (PMT superiori + bersaglio)
def simulate_event():
    # Generazione casuale di theta e phi
    phi = np.random.uniform(0, 2 * np.pi)  # Distribuzione uniforme in phi
    theta = np.arccos(np.random.uniform(0, 1)**(1/3)) # Distribuzione cos^2(theta)

    # PMT7
    x1 = np.random.uniform(0, 3*lunghezza_pla )
    y1 = np.random.uniform(0, 3*larghezza_pla)
    if not is_inside(x1, y1, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT7"]:
        return False

    # PMT4
    x2 = x1 - distanza_7_4 * np.tan(theta) * np.cos(phi)
    y2 = y1 - distanza_7_4 * np.tan(theta) * np.sin(phi)
    if not is_inside(x2, y2,lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT4"]:
        return False
    
    # PMT2
    x3 = x2 - distanza_4_2 * np.tan(theta) * np.cos(phi)
    y3 = y2 - distanza_4_2 * np.tan(theta) * np.sin(phi)
    if not is_inside(x3, y3, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT2"]:
        return False

    # Logica AND tra i PMT superiori e il bersaglio
    x89= x3 - distanza_2_b * np.tan(theta) * np.cos(phi)
    y89 = y3 - distanza_2_b* np.tan(theta) * np.sin(phi)
    return simulate_target_event(x89, y89,theta,phi)

# Simulazione Monte Carlo
def monte_carlo_simulation(duration, area):
    total_events = int(duration/60 * area)  # Numero totale di eventi da simulare
    coincident_events = 0
    for _ in range(total_events):
        if simulate_event():
            coincident_events += 1
    return coincident_events / duration  # Rate di coincidenza

# Esecuzione della simulazione
tempo_acquisizione = 1000 # secondi
area_totale = larghezza_pla * lunghezza_pla  # cm²
rate_coincidenza = monte_carlo_simulation(tempo_acquisizione, area_totale)
print(f"Rate di coincidenza  AND (sistema completo) per {tempo_acquisizione} s: {rate_coincidenza:.5f} eventi/s")

print(f"Quindi ho n eventi per ora {rate_coincidenza*3600:.5f} eventi")

Rate di coincidenza  AND (sistema completo) per 1000 s: 0.00600 eventi/s
Quindi ho n eventi per ora 21.60000 eventi


In [7]:
# Simulazione di un evento principale
def simulate_event_veto():
    
    # Generatore di numeri casuali
    rng = np.random.default_rng() #random seed
    
    # Selezione casuale per energia dei muoni (Genera l'energia dei muoni basandosi su una distribuzione piatta per energie <1 GeV.)
    muon_energy = rng.uniform(0, 1) * 1000 #da GeV a MeV
    if muon_energy > 100:
        return False  # I muoni con energia > 150 MeV non raggiungono il bersaglio
    #else:
     #   print(muon_energy)
    
    # Generazione casuale di theta e phi
    phi = rng.uniform(0, 2 * np.pi)  # Distribuzione uniforme in phi
    theta = np.arccos(rng.uniform(0, 1)**(1/3)) # Distribuzione cos^2(theta)

    # PMT7
    x1 = rng.uniform(0, 3*lunghezza_pla )
    y1 = rng.uniform(0, 3*larghezza_pla)
    if not is_inside(x1, y1, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT7"]:
        return False

    # PMT4
    x2 = x1 - distanza_7_4 * np.tan(theta) * np.cos(phi)
    y2 = y1 - distanza_7_4 * np.tan(theta) * np.sin(phi)
    if not is_inside(x2, y2,lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT4"]:
        return False
    
    # PMT2
    x3 = x2 - distanza_4_2 * np.tan(theta) * np.cos(phi)
    y3 = y2 - distanza_4_2 * np.tan(theta) * np.sin(phi)
    if not is_inside(x3, y3, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT2"]:
        return False

    # Logica AND tra i PMT superiori e il bersaglio
    x89= x3 - distanza_2_b * np.tan(theta) * np.cos(phi)
    y89 = y3 - distanza_2_b* np.tan(theta) * np.sin(phi)

    if not simulate_target_event(x89, y89,theta,phi):
         return False

    xb01 = x89- profondità_pmt_bers * np.tan(theta) * np.cos(phi)
    yb01 = y89 - profondità_pmt_bers * np.tan(theta) * np.sin(phi)
    
    # PMT1
    x4= xb01-distanza_b_1*np.tan(theta)*np.cos(phi)
    y4= yb01-distanza_b_1*np.tan(theta)*np.cos(phi)

    if is_inside(x4, y4, lunghezza_pla,larghezza_pla) and np.random.uniform(0, 1) < efficienze["PMT1"]:
         return False
    
    return True

# Simulazione Monte Carlo
def monte_carlo_simulation_and_veto(duration, area):
    total_events = int(duration/60 * area)  # Numero totale di eventi da simulare, con intensità di particelle di 1 particella per minuto su cm^2
    coincident_events = 0
    for _ in range(total_events):
        if simulate_event_veto():
            coincident_events += 1
    print(f"Totale coincidenze simulate per {coincident_events}")
    return coincident_events / duration  # Rate di coincidenza

# Esecuzione della simulazione
tempo_acquisizione = 10000 # secondi
area_totale = larghezza_pla * lunghezza_pla  # cm²
rate_coincidenza = monte_carlo_simulation_and_veto(tempo_acquisizione, area_totale)
print(f"Rate di coincidenza con VETO (sistema completo) per {tempo_acquisizione} s: {rate_coincidenza:.5f} eventi/s")

print(f"Quindi ho n eventi per ora {rate_coincidenza*3600:.5f} eventi")

Totale coincidenze simulate per 5
Rate di coincidenza con VETO (sistema completo) per 10000 s: 0.00050 eventi/s
Quindi ho n eventi per ora 1.80000 eventi


## Toy MonteCarlo STOP

## Efficienza intrinseca del VETO

In [8]:
def errore_prodotto(x,y,dx,dy):
    return np.sqrt((y*dx)**2+(x*dy)**2)
def errore_prodotto3(x,dx, y,dy, z, dz):
    return np.sqrt((x*y*dz)**2+(x*z*dy)**2+(dx*y*z)**2)

In [ ]:
# Simulazione di un evento principale
def accettanza_geometrica_34():
    """Calcolo della efficienza geometrica del PMT1 data la coincidenza tra 7 e 2."""
    n = 10000 #Numero di tentativi
    # Inizializza contatori
    successi = 0
    tentativi = 0
    for _ in range(n):
        # Generatore di numeri casuali
        rng = np.random.default_rng() #random seed
        
        # Generazione casuale di theta e phi
        phi = rng.uniform(0, 2 * np.pi)  # Distribuzione uniforme in phi
        theta = np.arccos(rng.uniform(0, 1)**(1/3)) # Distribuzione cos^2(theta)

        # PMT4
        x1 = rng.uniform(0, lunghezza_pla )
        y1 = rng.uniform(0, larghezza_pla)
        
        #PMT 3
        x2 = x1 - distanza_4_3 * np.tan(theta) * np.cos(phi)
        y2 = y1 - distanza_4_3 * np.tan(theta) * np.sin(phi)

        # PMT1
        x3= x2-(distanza_2_b+profondità_pmt_bers+distanza_b_1)*np.tan(theta)*np.cos(phi)
        y3= y2-(distanza_2_b+profondità_pmt_bers+distanza_b_1)*np.tan(theta)*np.cos(phi)

        # Verifica se x2 e y2 sono compresi nelle dimensioni del secondo scintillatore
        if 0 <= x2 <= lunghezza_pla and 0 <= y2 <= larghezza_pla:
            if 0 <= x3 <= lunghezza_pla and 0 <= y3 <= larghezza_pla:
                successi += 1
            tentativi+=1 #probabilità che passi dal terzo PMT data la doppia

    efficienza_geometrica = successi / tentativi
    errore = np.sqrt(efficienza_geometrica * (1 - efficienza_geometrica) / tentativi)
    return efficienza_geometrica, errore

In [11]:
triple_doppie1=0.13
errore1=0.03
triple_doppie2=0.32
errore2=0.03
triple_doppie3=0.38
errore3=0.04
triple_doppie4=0.73
errore4=0.04
triple_doppie7=0.32
errore7=0.03

acc_1, errore_acc_1=accettanza_geometrica_34()
print(f'Accettanza1: {acc_1}')

epsilon_intr1=triple_doppie1/acc_1
depsilon_intr1=errore_prodotto(triple_doppie1,acc_1,errore1,errore_acc_1)
print(f"Efficienza intrinseca PMT1 {epsilon_intr1:.2f} ± {depsilon_intr1:.2f}")

Accettanza1: 0.4592422502870264
Efficienza intrinseca PMT1 0.28 ± 0.01
